In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

text = """LangChain is a framework for developing applications powered by language models. It can be used for chatbots, Generative Question-Answering (GQA), summarization, and much more.
"""

loader = TextLoader(text)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [ ]:
from langchain.llms import OpenAI
from langchain.transformers import LLMGraphTransformer
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

llm = OpenAI(temperature=0)

llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(texts)

In [ ]:
from langchain.graph_stores import Neo4jGraphStore


graph_store = Neo4jGraphStore(url="neo4l://localhost:7687", username="neo4j", password="test")
graph_store.write_graph(graph_documents)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llam_index.core.response_synthesis import ResponseSynthesizer

graph_rag_retriever =KnowledgeGraphRAGRetriever(storage_context=graph_store.storage_context, verbose=True)
query_engine = RetrieverQueryEngine.from_args(graph_rag_retriever)

In [ ]:
def query_and_synthesize(query):
    retrieved_context = query_engine.query(query)
    response = response_synthesizer.synthesize(query, retrieved_context)
    print(f"Query:{query}")
    print(f"Answer: {response}")

response_synthesizer = ResponseSynthesizer(llm)

query_and_synthesize("Where does Sarah work?")

query_and_synthesize("Who works for prismaticAI?")

query_and_synthesize("Does Michael wprk for the same company as Sarah?")